In [123]:
import pandas as pd
import numpy as np

pd.set_option("display.precision", 2)

In [124]:
df = pd.read_csv("PartD_Prescriber_PUF_NPI_17.txt", sep='\t')
df.sample(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
839907,1720067093,SNOWDEN,JENNIFER,B,M.D.,F,I,1117 PERIMETER CTR,N404,ATLANTA,...,50.0,73.0,NaN,0.0,NaN,NaN,0.0,26.0,96.0,1.54
81514,1073044574,PITLICK,MITCHELL,NaN,NaN,M,I,200 1ST ST SW,NaN,ROCHESTER,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,1.73
835095,1710942842,ARNESON,RICHARD,J,OD,M,I,2119 W 12TH ST,NaN,HASTINGS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158488,1134385651,DESENZE,PHILIP,S,NaN,M,I,540 E MCNAB RD,NaN,POMPANO BEACH,...,12.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.33
682961,1588774848,RISNER,DARRELL,H,DMD,M,I,8 DOGWOOD ST,NaN,HYDEN,...,31.0,55.0,0.0,0.0,0.0,0.0,0.0,37.0,18.0,1.15


In [125]:
print(df.shape)

(1162898, 84)


In [132]:
len(pd.unique(df['npi']))

1162898

**Question:**
In 2017, what was the average number of beneficiaries per provider? Due to the suppression of data for those with few beneficiaries, we can only include those with more than 10 beneficiaries.

In [127]:
total_beneficiaries = df['beneficiary_female_count'] + df['beneficiary_male_count']
total_beneficiaries[total_beneficiaries > 10].mean()

192.62371485069525

In [128]:
df['total_claim_count']

0           677
1          1946
2            55
3            18
4            90
           ... 
1162893     270
1162894      59
1162895      15
1162896      81
1162897     256
Name: total_claim_count, Length: 1162898, dtype: int64

In [129]:
df['total_day_supply']

0          14788
1          58605
2            430
3            150
4           2681
           ...  
1162893     5566
1162894     1866
1162895      288
1162896     1879
1162897     7945
Name: total_day_supply, Length: 1162898, dtype: int64

**Question:**
For each provider, estimate the length of the average prescription from the total_day_supply and total_claim_count. What is the median, in days, of the distribution of this value across all providers?

In [130]:
(df['total_day_supply'] / df['total_claim_count']).median()

29.7125748502994

In [ ]:
df['specialty_description']

**Question:** 
Work out for each Specialty the fraction of drug claims that are for brand-name drugs. Include only providers for whom the relevant information has not been suppressed, and consider only specialties with at least 1000 total claims. What is the standard deviation of these fractions?

In [133]:
not_suppressed_providers = df[df['brand_claim_count'].notnull()]
specialty_brand_claims = (not_suppressed_providers.groupby(['specialty_description']))[['brand_claim_count']].agg('sum')
total_brand_claims = specialty_brand_claims['brand_claim_count'].sum()
sbc_grtr_than_th = specialty_brand_claims[specialty_brand_claims['brand_claim_count'] > 1000]
(sbc_grtr_than_th['brand_claim_count'] / total_brand_claims).std()


0.041203014890554175

**Question:**
Find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions in each state. Assume that each beneficiary attends only a single provider. What is the difference between the largest and smallest ratios?

In [45]:
state_opioid_bene_count = (df.groupby(['nppes_provider_state']))[['opioid_bene_count']].agg('sum')
state_antibiotic_bene_count = (df.groupby(['nppes_provider_state']))[['antibiotic_bene_count']].agg('sum')
max_ratio = (state_opioid_bene_count['opioid_bene_count'] / state_antibiotic_bene_count['antibiotic_bene_count']).max()
min_ratio = (state_opioid_bene_count['opioid_bene_count'] / state_antibiotic_bene_count['antibiotic_bene_count']).min()

max_ratio - min_ratio



0.6331097230267027

**Question:**
For each provider where the relevant columns are not suppressed, work out the fraction of claims for beneficiaries age 65 and older, as well as the fraction of claims for beneficiaries with a low-income subsidy. What is the Pearson correlation coefficient between these values?

In [134]:
unsuppressed_df = df[df['total_claim_count_ge65'].notnull()]
unsuppressed_df = df[df['lis_claim_count'].notnull()]
sum_ge65_claims = unsuppressed_df['total_claim_count_ge65'].sum()
sum_lis_claims = unsuppressed_df['lis_claim_count'].sum()
frac_ge65_claims = unsuppressed_df['total_claim_count_ge65'] / sum_ge65_claims
frac_lis_claims = unsuppressed_df['lis_claim_count'] / sum_lis_claims
frac_ge65_claims.corr(frac_lis_claims)

0.8067444425921829

**Question:**
Let's find which states have surprisingly high supply of opioids, conditioned on specialty. Work out the average length of an opioid prescription for each provider. For each (state, specialty) pair with at least 100 providers, calculate the average of this value across all providers. Then find the ratio of this value to an equivalent quantity calculated from providers in each specialty across all states. What is the largest such ratio?

In [105]:
df['avg_length_op_pre'] = (df['opioid_day_supply'] / df['opioid_claim_count'])
st_sp_atleast_hund_prvdrs = (df.groupby(['nppes_provider_state', 'specialty_description'])).filter(lambda x: len(x) >= 100)
avg_length_op_pre_st_sp = st_sp_atleast_hund_prvdrs.groupby(['nppes_provider_state', 'specialty_description'])['avg_length_op_pre'].mean().reset_index()
avg_length_op_pre_sp = df.groupby(['specialty_description'])['avg_length_op_pre'].mean().reset_index()
max = 0
for index, row in avg_length_op_pre_st_sp.iterrows():
    ratio = row['avg_length_op_pre'] / (avg_length_op_pre_sp[avg_length_op_pre_sp['specialty_description'] == row['specialty_description']]['avg_length_op_pre'])
    if ratio.iloc[0] > max:
        max = ratio.iloc[0]
max

2.304597636251375

In [107]:
df16 = pd.read_csv("PartD_Prescriber_PUF_NPI_16.txt", sep='\t')
df16.sample(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
971355,1851699060,COOLEY,ALISHA,J,APRN,F,I,56 TOWER CIR,NaN,SOMERSET,...,197.0,NaN,NaN,0.0,NaN,0.0,NaN,357.0,174.0,1.13
550338,1487739298,MAYER,CHARLES,J,NaN,M,I,UNIVERSITY OF WASHINGTON MEDICAL CTR,1959 NE PACIFIC ST,SEATTLE,...,121.0,163.0,NaN,17.0,NaN,NaN,NaN,189.0,12.0,0.80
1051136,1922456870,LUFTIG,DANIEL,NaN,MD,M,I,777 N RAYMOND ST,NaN,BOISE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
695439,1619248325,FREUND,KELLY,D,PA-C,F,I,1901 N MAIZE RD,NaN,WICHITA,...,70.0,NaN,NaN,0.0,NaN,0.0,NaN,155.0,23.0,1.01
590804,1528041381,MOORE,BRANDI,N,PA,F,I,199 HOSPITAL DR,SUITE 5,GALAX,...,117.0,280.0,NaN,NaN,NaN,0.0,NaN,192.0,103.0,1.18


**Question:**
For each provider for whom the information is not suppressed, figure out the average cost per day of prescriptions in both 2016 and 2017. Use this to estimate the inflation rate for daily prescription costs per provider. What is the average inflation rate across all providers?

In [141]:
df17 = df
cost_mean_17 = df17[df17['total_drug_cost'].notnull()]['total_drug_cost'].mean()
cost_mean_16 = df16[df16['total_drug_cost'].notnull()]['total_drug_cost'].mean()
inflation_rate = ((cost_mean_17 - cost_mean_16) / cost_mean_17)
inflation_rate

0.029856522335159293

**Question:**
Consider all providers with a defined specialty in both years. Find the fraction of providers who left each specialty between 2016 and 2017. What is the largest such fraction, when considering specialties with at least 1000 proviers in 2016? Note that some specialties have a fraction of 1 due to specialty name changes between 2016 and 2017; disregard these specialties in calculating your answer.

In [152]:
df16_sp_atleast_th = df16.groupby(['specialty_description']).filter(lambda x: len(x) >= 1000)
total_providers = len(df16_sp_atleast_th['npi'])

max_fraction = 0
for specialty in pd.unique(df16_sp_atleast_th['specialty_description']):
    providers_16 = df16[df16['specialty_description'] == specialty]['npi']
    providers_17 = df17[df17['specialty_description'] == specialty]['npi']
    total_left = len(np.setdiff1d(providers_16, providers_17))
    fraction = total_left / total_providers
    if fraction > max_fraction:
        max_fraction = fraction
max_fraction
    

0.01828027238234038